In [1]:
# Imports
import numpy as np
import pandas as pd
from collections import Counter
import hashlib
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

# Data Preparation for Team Analysis

In [27]:
# Import necessary libraries and load team data
from collections import Counter
import hashlib
from sklearn.preprocessing import LabelEncoder

class MatchIDGenerator:
    def __init__(self):
        self.match_id_dict = {}

    def generate_match_id(self, row):
        match_info = f"{row['Ground']}_{row['Start Date']}"
        match_id = hashlib.sha256(match_info.encode()).hexdigest()
        self.match_id_dict[match_id] = (row['Ground'], row['Start Date'])
        return match_id

# Load team data
df_team = pd.read_csv('clean_team.csv', parse_dates=["Start Date"])
df_team['ScoreDescending'] = pd.to_numeric(df_team['ScoreDescending'], errors='coerce').astype('Int64')
df_team['Wickets'] = df_team['Wickets'].astype('Int64')
df_team['Start Date'] = pd.to_datetime(df_team['Start Date'])

# Create a dictionary to store the count of teams at each ground
ground_teams_count = {}

# Iterate through the DataFrame to count the occurrences of each team at each ground
for index, row in df_team.iterrows():
    ground = row['Ground']
    teams = [row['Team'], row['Opposition']]

    if ground not in ground_teams_count:
        ground_teams_count[ground] = Counter(teams)
    else:
        ground_teams_count[ground] += Counter(teams)

# Create a new column "Host" based on the team with the maximum count at each ground
df_team['Host'] = df_team['Ground'].apply(lambda ground: ground_teams_count[ground].most_common(1)[0][0] if ground in ground_teams_count else '')


In [28]:
df_team.head(10)

,Team,ScoreDescending,Overs,RPO,Lead,Inns,Result,Opposition,Ground,Start Date,Declared,Wickets,Host
0,Sri Lanka,952,271.000000,3.51,415,2,draw,India,Colombo (RPS),1997-08-02,1,6,Sri Lanka
1,England,903,335.333333,2.69,903,1,won,Australia,The Oval,1938-08-20,1,7,England
2,England,849,258.333333,3.28,849,1,draw,West Indies,Kingston,1930-04-03,0,10,West Indies
3,West Indies,790,208.166667,3.79,462,2,won,Pakistan,Kingston,1958-02-26,1,3,West Indies
4,Pakistan,765,248.833333,3.07,121,2,draw,Sri Lanka,Karachi,2009-02-21,1,6,Pakistan
5,Sri Lanka,760,202.666667,3.75,334,2,draw,India,Ahmedabad,2009-11-16,1,7,India
6,India,759,190.666667,3.98,282,2,won,England,Chennai,2016-12-16,1,7,India
7,Australia,758,245.666667,3.08,401,2,won,West Indies,Kingston,1955-06-11,1,8,West Indies
8,Sri Lanka,756,185.166667,4.08,587,2,won,South Africa,Colombo (SSC),2006-07-27,1,5,Sri Lanka
9,West Indies,751,202.000000,3.71,751,1,draw,England,St John's,2004-04-10,1,5,West Indies


# Filtering team data for matches since 1985

In [29]:
# Handling missing hosts and filtering data

grounds_without_host = df_team[df_team['Host'] == '']
print("Grounds without a host:")
print(grounds_without_host[['Ground', 'Host']])

# Filter the DataFrame for data from 1985-01-01 onwards
df_team = df_team[df_team['Start Date'] >= '1985-01-01']


Grounds without a host:
Empty DataFrame
Columns: [Ground, Host]
Index: []


# Loading and Processing Batting and Bowling Data

In [30]:
# Process batting and bowling data

def process_cricket_data(df):
    df.rename(columns={'Country': 'Team'}, inplace=True)
    df['Start Date'] = pd.to_datetime(df['Start Date'])
    df = df[df['Start Date'] >= '1985-01-01']
    return df

# Load and process batting data
df_batting = pd.read_csv('clean_batting.csv', parse_dates=["Start Date"])
df_batting = process_cricket_data(df_batting)
df_batting = df_batting.drop('Mins', axis=1)

# Load and process bowling data
df_bowling = pd.read_csv('clean_bowling.csv', parse_dates=["Start Date"])
df_bowling = process_cricket_data(df_bowling)
df_bowling = df_bowling.drop('BPO', axis=1)


C:\Users\Student3\AppData\Local\Temp\ipykernel_11296\8760988.py:10: DtypeWarning:

Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.



In [6]:
df_batting.to_csv('bat.csv', index=False)
df_bowling.to_csv('bowl.csv', index=False)
df_team.to_csv('team.csv', index=False)

# Unifying and Encoding Match IDs Across DataFrames

In [31]:
# Match ID handling and label encoding

# Initialize MatchIDGenerator
match_id_generator = MatchIDGenerator()

# Apply the function to create the 'Match ID' column in 'team' DataFrame
df_team['Match ID'] = df_team.apply(match_id_generator.generate_match_id, axis=1)

# Display the updated 'team' DataFrame
df_team.head()

# Apply the function to create the 'Match ID' column in 'batting' DataFrame
df_batting['Match ID'] = df_batting.apply(match_id_generator.generate_match_id, axis=1)

# Display the updated 'batting' DataFrame
df_batting.head()

# Apply the function to create the 'Match ID' column in 'bowling' DataFrame
df_bowling['Match ID'] = df_bowling.apply(match_id_generator.generate_match_id, axis=1)

# Display the updated 'bowling' DataFrame
df_bowling.head()

# Combine all Match IDs from the four dataframes
all_match_ids = pd.concat([df_team['Match ID'], df_batting['Match ID'], df_bowling['Match ID']], axis=0)

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit the label encoder on the combined unique Match IDs
label_encoder.fit(all_match_ids)

# Transform the Match IDs in all dataframes to numerical values
df_team['NumericMatchID'] = label_encoder.transform(df_team['Match ID'])
df_batting['NumericMatchID'] = label_encoder.transform(df_batting['Match ID'])
df_bowling['NumericMatchID'] = label_encoder.transform(df_bowling['Match ID'])

df_team = df_team.drop('Match ID', axis=1)
df_batting = df_batting.drop('Match ID', axis=1)
df_bowling = df_bowling.drop('Match ID', axis=1)


In [32]:
filtered_rowsbat = df_batting[df_batting['Start Date'] == '2013-01-02']
filtered_rowsbat.head(50)

,Player,RunsDescending,BF,4s,6s,SR,Inns,Opposition,Ground,Start Date,Not Out,Team,NumericMatchID
3379,DG Brownlie,109,186,13,2,58.60,3,South Africa,Cape Town,2013-01-02,0,New Zealand,926
3636,AN Petersen,106,176,11,1,60.22,2,New Zealand,Cape Town,2013-01-02,0,South Africa,926
9496,AB de Villiers,67,129,8,0,51.93,2,New Zealand,Cape Town,2013-01-02,0,South Africa,926
9748,HM Amla,66,74,9,0,89.18,2,New Zealand,Cape Town,2013-01-02,0,South Africa,926
11534,JH Kallis,60,89,5,2,67.41,2,New Zealand,Cape Town,2013-01-02,0,South Africa,926
14658,BB McCullum,51,103,7,0,49.51,3,South Africa,Cape Town,2013-01-02,0,New Zealand,926
18751,BJ Watling,42,151,4,0,27.81,3,South Africa,Cape Town,2013-01-02,0,New Zealand,926
33843,JEC Franklin,22,59,2,0,37.28,3,South Africa,Cape Town,2013-01-02,0,New Zealand,926
34935,D Elgar,21,52,2,0,40.38,2,New Zealand,Cape Town,2013-01-02,0,South Africa,926
42811,F du Plessis,15,28,2,0,53.57,2,New Zealand,Cape Town,2013-01-02,0,South Africa,926


In [33]:
filtered_rowsbowl = df_bowling[df_bowling['Start Date'] == '2013-01-02']
filtered_rowsbowl.head(50)

,Player,Overs,Mdns,Runs,WktsDescending,Econ,Inns,Opposition,Ground,Start Date,Team,NumericMatchID
1203,VD Philander,6.000000,3,7,5,1.16,1,New Zealand,Cape Town,2013-01-02,South Africa,926
6622,M Morkel,6.000000,2,14,3,2.33,1,New Zealand,Cape Town,2013-01-02,South Africa,926
9366,CS Martin,19.333333,4,63,3,3.25,2,South Africa,Cape Town,2013-01-02,New Zealand,926
9660,DW Steyn,30.000000,6,67,3,2.23,3,New Zealand,Cape Town,2013-01-02,South Africa,926
10337,TA Boult,21.000000,2,78,3,3.71,2,South Africa,Cape Town,2013-01-02,New Zealand,926
12498,DW Steyn,7.333333,2,18,2,2.45,1,New Zealand,Cape Town,2013-01-02,South Africa,926
13495,JH Kallis,11.166667,3,31,2,2.77,3,New Zealand,Cape Town,2013-01-02,South Africa,926
18014,VD Philander,24.000000,8,76,2,3.16,3,New Zealand,Cape Town,2013-01-02,South Africa,926
25469,RJ Peterson,16.000000,6,42,1,2.62,3,New Zealand,Cape Town,2013-01-02,South Africa,926
26620,JEC Franklin,14.000000,1,50,1,3.57,2,South Africa,Cape Town,2013-01-02,New Zealand,926


In [34]:
df_batting.drop(df_batting[(df_batting['RunsDescending'] == 0) & (df_batting['BF'] == 0)].index, inplace=True)

In [35]:
count_dash = (df_team['Inns'] == '-').sum()
print("Number of rows with '-' in the 'Inns' column:", count_dash)


Number of rows with '-' in the 'Inns' column: 0


In [36]:
df_team['Outcome'] = df_team['Result'].map({'won': 'Win', 'lost': 'Loss', 'draw': 'Draw'})


In [37]:
team_outcome_counts = df_team.groupby(['Team', 'Outcome']).size().unstack(fill_value=0)


In [38]:
team_outcome_counts['Win-Loss Ratio'] = team_outcome_counts['Win'] / team_outcome_counts['Loss'].replace(0, 1)


In [39]:
sorted_teams = team_outcome_counts.sort_values(by='Win-Loss Ratio', ascending=False)


In [40]:
fig = px.bar(sorted_teams, x=sorted_teams.index, y='Win-Loss Ratio', title='Teams Win/Loss Ratio',
             labels={'index': 'Team', 'Win-Loss Ratio': 'Win/Loss Ratio'})
fig.show()


In [41]:
# Map 'Result' to 'Outcome' with 'Win', 'Not Win' (draw + lost)
df_team['Outcome'] = df_team['Result'].map({'won': 'Win', 'lost': 'Not Win', 'draw': 'Not Win'})

# Group by 'Team' and 'Outcome' and count occurrences
team_outcome_counts = df_team.groupby(['Team', 'Outcome']).size().unstack(fill_value=0)

# Calculate win ratio
team_outcome_counts['Win Ratio'] = team_outcome_counts['Win'] / (team_outcome_counts['Win'] + team_outcome_counts['Not Win'])

# Sort teams based on win ratio
sorted_teams = team_outcome_counts.sort_values(by='Win Ratio', ascending=False)

# Create bar plot
fig = px.bar(sorted_teams, x=sorted_teams.index, y='Win Ratio', title='Teams Win/Not Win Ratio',
             labels={'index': 'Team', 'Win Ratio': 'Win/Not Win Ratio'})

# Show plot
fig.show()


In [42]:
df_batting.head()

,Player,RunsDescending,BF,4s,6s,SR,Inns,Opposition,Ground,Start Date,Not Out,Team,NumericMatchID
0,BC Lara,400,582,43,4,68.72,1,England,St John's,2004-04-10,1,West Indies,658
1,ML Hayden,380,437,38,11,86.95,1,Zimbabwe,Perth,2003-10-09,0,Australia,1174
2,BC Lara,375,538,45,0,69.70,1,England,St John's,1994-04-16,0,West Indies,335
3,DPMD Jayawardene,374,572,43,1,65.38,2,South Africa,Colombo (SSC),2006-07-27,0,Sri Lanka,294
6,ST Jayasuriya,340,578,36,2,58.82,2,India,Colombo (RPS),1997-08-02,0,Sri Lanka,648


In [43]:
df_batting.dtypes

Player                    object
RunsDescending             int64
BF                         int64
4s                         int64
6s                         int64
SR                       float64
Inns                      object
Opposition                object
Ground                    object
Start Date        datetime64[ns]
Not Out                    int64
Team                      object
NumericMatchID             int32
dtype: object

In [44]:
# Calculate the batting average for each team
batting_avg_df = df_batting.groupby('Team').agg(
    TotalRuns=('RunsDescending', 'sum'),
    TotalOuts=('Not Out', lambda x: (x == 0).sum())
)

batting_avg_df['BattingAverage'] = 11*batting_avg_df['TotalRuns'] / batting_avg_df['TotalOuts']

# Plotting with Plotly
fig = px.bar(batting_avg_df, x=batting_avg_df.index, y='BattingAverage',
             labels={'BattingAverage': 'Batting Average'},
             title='Batting Average for Each Team')

fig.show()

In [45]:
# Calculate batting average, total runs, matches played, and highest score per player per year
df_avg = df_batting.groupby(['Player', df_batting['Start Date'].dt.year])\
    .apply(lambda group: pd.Series({
        'Batting Average': group['RunsDescending'].sum() / (1 - group['Not Out']).sum() if (1 - group['Not Out']).sum() > 0 else 0,
        'Total Runs': group['RunsDescending'].sum(),
        'Matches Played': group['NumericMatchID'].nunique(),
        'Highest Score': group['RunsDescending'].max()
    }))\
    .reset_index()

# Prompt user to enter a player
player_name = input("Enter player name: ")

# Filter DataFrame for the specified player
df_player = df_avg[df_avg['Player'] == player_name]

# Plot the batting average vs. years using Plotly Express
fig = px.line(df_player, x='Start Date', y='Batting Average', 
              hover_data={'Total Runs': True, 'Matches Played': True, 'Highest Score': True}, 
              title=f'{player_name} Batting Performance Over the Years',
              labels={'Batting Average': 'Batting Average', 'Start Date': 'Year'})
fig.show()

Enter player name: Q de Kock


In [59]:
df_batting['Year'] = df_batting['Start Date'].dt.year
df_batting.sort_values(by=['Player', 'Year'], inplace=True)

# Group by Player and Year, then calculate yearly statistics
yearly_stats = df_batting.groupby(['Player', 'Year']).apply(lambda group: pd.Series({
    'Total Runs': group['RunsDescending'].sum(),
    'Outs': (1 - group['Not Out']).sum(),
    'Matches Played': group['NumericMatchID'].nunique(),
    'Highest Score': group['RunsDescending'].max()
}))

# Reset index to handle data easier
yearly_stats.reset_index(inplace=True)

# Initialize columns for cumulative statistics
yearly_stats['Cumulative Runs'] = 0
yearly_stats['Cumulative Outs'] = 0
yearly_stats['Cumulative Matches Played'] = 0
yearly_stats['Cumulative Highest Score'] = 0

# Calculate cumulative values manually to maintain state across years
for player in yearly_stats['Player'].unique():
    player_data = yearly_stats[yearly_stats['Player'] == player]
    cum_runs = cum_outs = cum_matches = 0
    max_score = 0
    for index, row in player_data.iterrows():
        cum_runs += row['Total Runs']
        cum_outs += row['Outs']
        cum_matches += row['Matches Played']
        max_score = max(max_score, row['Highest Score'])
        
        yearly_stats.loc[index, 'Cumulative Runs'] = cum_runs
        yearly_stats.loc[index, 'Cumulative Outs'] = cum_outs
        yearly_stats.loc[index, 'Cumulative Matches Played'] = cum_matches
        yearly_stats.loc[index, 'Cumulative Highest Score'] = max_score

# Calculate cumulative batting average
yearly_stats['Cumulative Batting Average'] = yearly_stats['Cumulative Runs'] / yearly_stats['Cumulative Outs']

# Prompt user to enter a player
player_name = input("Enter player name: ")

# Filter DataFrame for the specified player
df_player = yearly_stats[yearly_stats['Player'] == player_name]

# Plot the cumulative batting average vs. years using Plotly Express
fig = px.line(df_player, x='Year', y='Cumulative Batting Average', 
       hover_data={'Cumulative Runs': True, 'Cumulative Matches Played': True, 'Cumulative Highest Score': True}, 
       title=f'{player_name} Cumulative Batting Average Over the Years',
       labels={'Cumulative Batting Average': 'Cumulative Batting Average', 'Year': 'Year'})
fig.show()


Enter player name: JH Kallis


In [60]:
# Calculate bowling average, total runs conceded, matches played, and best bowling figures per player per year
df_avg_bowling = df_bowling.groupby(['Player', df_bowling['Start Date'].dt.year])\
    .apply(lambda group: pd.Series({
        'Bowling Average': group['Runs'].sum() / group['WktsDescending'].sum() if group['WktsDescending'].sum() > 0 else 0,
        'Total Wickets Taken': group['WktsDescending'].sum(),
        'Matches Played': group['NumericMatchID'].nunique(),
        'Best Bowling Figures': group.sort_values(by=['WktsDescending', 'Runs'], ascending=[False, True]).iloc[0]['WktsDescending'].astype(int).astype(str) + "-" + group.sort_values(by=['WktsDescending', 'Runs'], ascending=[False, True]).iloc[0]['Runs'].astype(int).astype(str)
    }))\
    .reset_index()

# Prompt user to enter a player
player_name = input("Enter player name: ")

# Filter DataFrame for the specified player
df_player_bowling = df_avg_bowling[df_avg_bowling['Player'] == player_name]

# Plot the bowling average vs. years using Plotly Express
fig = px.line(df_player_bowling, x='Start Date', y='Bowling Average', 
              hover_data={'Total Wickets Taken': True, 'Matches Played': True, 'Best Bowling Figures': True}, 
              title=f'{player_name} Bowling Performance Over the Years',
              labels={'Bowling Average': 'Bowling Average', 'Start Date': 'Year'})
fig.show()


Enter player name: DW Steyn


In [68]:
# Calculate yearly bowling statistics
df_bowling['Year'] = df_bowling['Start Date'].dt.year
df_avg_bowling = df_bowling.groupby(['Player', 'Year']).apply(lambda group: pd.Series({
    'Total Runs Conceded': group['Runs'].sum(),
    'Total Wickets Taken': group['WktsDescending'].sum(),
    'Matches Played': group['NumericMatchID'].nunique(),
    'Best Bowling Figures': group.sort_values(by=['WktsDescending', 'Runs'], ascending=[False, True]).iloc[0]['WktsDescending'].astype(int).astype(str) + "-" + group.sort_values(by=['WktsDescending', 'Runs'], ascending=[False, True]).iloc[0]['Runs'].astype(int).astype(str)
})).reset_index()

# Initialize columns for cumulative statistics
df_avg_bowling['Cumulative Runs Conceded'] = 0
df_avg_bowling['Cumulative Wickets Taken'] = 0
df_avg_bowling['Cumulative Matches Played'] = 0
df_avg_bowling['Cumulative Best Bowling Figures'] = "0-0"
df_avg_bowling['Cumulative Bowling Average'] = 0.0

# Calculate cumulative values manually to maintain state across years
for player in df_avg_bowling['Player'].unique():
    player_data = df_avg_bowling[df_avg_bowling['Player'] == player].sort_values('Year')
    cum_runs = cum_wickets = cum_matches = 0
    best_figures = "0-0"
    
    for index, row in player_data.iterrows():
        cum_runs += row['Total Runs Conceded']
        cum_wickets += row['Total Wickets Taken']
        cum_matches += row['Matches Played']
        
        # Update best bowling figures if the new one is better
        if int(row['Best Bowling Figures'].split("-")[0]) > int(best_figures.split("-")[0]) or (int(row['Best Bowling Figures'].split("-")[0]) == int(best_figures.split("-")[0]) and int(row['Best Bowling Figures'].split("-")[1]) < int(best_figures.split("-")[1])):
            best_figures = row['Best Bowling Figures']
        
        # Update cumulative statistics
        df_avg_bowling.loc[index, 'Cumulative Runs Conceded'] = cum_runs
        df_avg_bowling.loc[index, 'Cumulative Wickets Taken'] = cum_wickets
        df_avg_bowling.loc[index, 'Cumulative Matches Played'] = cum_matches
        df_avg_bowling.loc[index, 'Cumulative Best Bowling Figures'] = best_figures
        
        # Calculate cumulative bowling average if there are wickets taken
        if cum_wickets > 0:
            df_avg_bowling.loc[index, 'Cumulative Bowling Average'] = cum_runs / cum_wickets
        else:
            df_avg_bowling.loc[index, 'Cumulative Bowling Average'] = 0

# Prompt user to enter a player
player_name = input("Enter player name: ")

# Filter DataFrame for the specified player
df_player_bowling = df_avg_bowling[df_avg_bowling['Player'] == player_name]

# Plot the cumulative bowling performance vs. years using Plotly Express
fig = px.line(df_player_bowling, x='Year', y='Cumulative Bowling Average',
              hover_data={'Cumulative Runs Conceded': True, 'Cumulative Wickets Taken': True, 'Cumulative Matches Played': True, 'Cumulative Best Bowling Figures': True},
              title=f'{player_name} Cumulative Bowling Performance Over the Years',
              labels={'Cumulative Bowling Average': 'Cumulative Bowling Average', 'Year': 'Year'})
fig.show()


Enter player name: K Rabada


In [65]:
# Calculate batting average, total runs, matches played, and highest score per player per year
df_avg = df_batting.groupby(['Player', df_batting['Start Date'].dt.year])\
    .apply(lambda group: pd.Series({
        'Batting Average': group['RunsDescending'].sum() / (1 - group['Not Out']).sum() if (1 - group['Not Out']).sum() > 0 else 0,
        'Total Runs': group['RunsDescending'].sum(),
        'Matches Played': group['NumericMatchID'].nunique(),
        'Highest Score': group['RunsDescending'].max()
    }))\
    .reset_index()

# Prompt user to enter two players
player1_name = input("Enter first player's name: ")
player2_name = input("Enter second player's name: ")

# Filter DataFrame for the specified players
df_players = df_avg[df_avg['Player'].isin([player1_name, player2_name])]

# Plot the batting average vs. years for both players using Plotly Express
fig = px.line(df_players, x='Start Date', y='Batting Average', color='Player',
              hover_data={'Total Runs': True, 'Matches Played': True, 'Highest Score': True}, 
              title=f'{player1_name} vs. {player2_name} Batting Performance Over the Years',
              labels={'Batting Average': 'Batting Average', 'Start Date': 'Year', 'Player': 'Player'})
fig.show()

Enter first player's name: AB de Villiers
Enter second player's name: HM Amla


In [26]:
# Calculate bowling average, total wickets taken, matches played, and best bowling figures per player per year
df_avg_bowling = df_bowling.groupby(['Player', df_bowling['Start Date'].dt.year])\
    .apply(lambda group: pd.Series({
        'Bowling Average': group['Runs'].sum() / group['WktsDescending'].sum() if group['WktsDescending'].sum() > 0 else 0,
        'Total Wickets Taken': group['WktsDescending'].sum(),
        'Matches Played': group['NumericMatchID'].nunique(),
        'Best Bowling Figures': group.sort_values(by=['WktsDescending', 'Runs'], ascending=[False, True]).iloc[0]['WktsDescending'].astype(int).astype(str) + "-" + group.sort_values(by=['WktsDescending', 'Runs'], ascending=[False, True]).iloc[0]['Runs'].astype(int).astype(str)
    }))\
    .reset_index()

# Prompt user to enter two players
player1_name = input("Enter first player's name: ")
player2_name = input("Enter second player's name: ")

# Filter DataFrame for the specified players
df_players_bowling = df_avg_bowling[df_avg_bowling['Player'].isin([player1_name, player2_name])]

# Plot the bowling average vs. years for both players using Plotly Express
fig = px.line(df_players_bowling, x='Start Date', y='Bowling Average', color='Player',
              hover_data={'Total Wickets Taken': True, 'Matches Played': True, 'Best Bowling Figures': True}, 
              title=f'{player1_name} vs. {player2_name} Bowling Performance Over the Years',
              labels={'Bowling Average': 'Bowling Average', 'Start Date': 'Year', 'Player': 'Player'})
fig.show()


Enter first player's name: DW Steyn
Enter second player's name: M Morkel


In [81]:
# Step 1: Filter df_batting for player and minimum score
min_score = int(input("Enter the minimum score: "))
player_name = input("Enter the player's name: ")

player_matches = df_batting[(df_batting['Player'] == player_name) & (df_batting['RunsDescending'] >= min_score)]

# Step 2: Extract unique matches where the player scored the minimum score or more
player_matches_ids = player_matches['NumericMatchID'].unique()

# Step 3: Filter df_team for matches where the player's team played
team_matches = df_team[df_team['NumericMatchID'].isin(player_matches_ids)]

# Step 4: Count the number of matches won, lost, and drawn by the player's team
team_won_matches_count = team_matches[(team_matches['Team'] == player_matches.iloc[0]['Team']) & (team_matches['Result'] == 'won')]['NumericMatchID'].nunique()
team_lost_matches_count = team_matches[(team_matches['Team'] == player_matches.iloc[0]['Team']) & (team_matches['Result'] == 'lost')]['NumericMatchID'].nunique()
team_drawn_matches_count = team_matches[(team_matches['Team'] == player_matches.iloc[0]['Team']) & (team_matches['Result'] == 'draw')]['NumericMatchID'].nunique()

# Step 5: Calculate the winning percentage
total_matches = len(player_matches_ids)

if total_matches == 0:
    print(f"No matches found for {player_name} with runs greater than or equal to {min_score}.")
else:
    winning_percentage = (team_won_matches_count / total_matches) * 100
    losing_percentage = (team_lost_matches_count / total_matches) * 100
    drawing_percentage = (team_drawn_matches_count / total_matches) * 100
    print(f"Total matches where {player_name} scored {min_score} or more: {total_matches}")
    print(f"Matches won by {player_name}'s team: {team_won_matches_count}")
    print(f"Matches lost by {player_name}'s team: {team_lost_matches_count}")
    print(f"Matches drawn by {player_name}'s team: {team_drawn_matches_count}")
    print(f"The winning percentage for {player_name}'s team (minimum score: {min_score}) is {winning_percentage:.2f}%")
    print(f"The losing percentage for {player_name}'s team (minimum score: {min_score}) is {losing_percentage:.2f}%")
    print(f"The drawing percentage for {player_name}'s team (minimum score: {min_score}) is {drawing_percentage:.2f}%")


Enter the minimum score: 100
Enter the player's name: HM Amla
Total matches where HM Amla scored 100 or more: 27
Matches won by HM Amla's team: 16
Matches lost by HM Amla's team: 3
Matches drawn by HM Amla's team: 8
The winning percentage for HM Amla's team (minimum score: 100) is 59.26%
The losing percentage for HM Amla's team (minimum score: 100) is 11.11%
The drawing percentage for HM Amla's team (minimum score: 100) is 29.63%


In [29]:
# Prompt the user to enter the player's name
player_name = input("Enter the player's name: ")

# Filter data for the player
player_data = df_batting[df_batting['Player'] == player_name]

# Calculate the overall average for the player
total_runs = player_data['RunsDescending'].sum()
times_out = (1 - player_data['Not Out']).sum()  # Count of times the player was out

if times_out > 0:
    overall_average = total_runs / times_out
else:
    overall_average = 0

print(f"{player_name}'s overall batting average is:", overall_average)

# Calculate the average against each opposition
opposition_averages = {}
for opposition, opposition_data in player_data.groupby('Opposition'):
    opposition_total_runs = opposition_data['RunsDescending'].sum()
    opposition_times_out = (1 - opposition_data['Not Out']).sum()
    if opposition_times_out > 0:
        opposition_average = opposition_total_runs / opposition_times_out
    else:
        opposition_average = 0
    opposition_averages[opposition] = opposition_average

print(f"\n{player_name}'s batting averages against each opposition:")
for opposition, average in opposition_averages.items():
    print(f"{opposition}: {average:.2f}")


Enter the player's name: Q de Kock
Q de Kock's overall batting average is: 38.8235294117647

Q de Kock's batting averages against each opposition:
Australia: 38.93
Bangladesh: 18.00
England: 41.65
India: 20.14
New Zealand: 53.57
Pakistan: 37.12
Sri Lanka: 36.68
West Indies: 127.50
Zimbabwe: 52.50


In [30]:
# Prompt the user to enter the player's name
player_name = input("Enter the player's name: ")

# Filter data for the player
player_data = df_batting[df_batting['Player'] == player_name]

# Create a dictionary to store total runs and times out for each country
country_totals = {}
country_counts = {}

# Iterate through each match the player played
for index, match in player_data.iterrows():
    match_id = match['NumericMatchID']
    # Find the host country for the match
    host_country = df_team[df_team['NumericMatchID'] == match_id]['Host'].iloc[0]
    # Add runs and times out to the corresponding country in the dictionary
    if host_country not in country_totals:
        country_totals[host_country] = 0
        country_counts[host_country] = 0
    country_totals[host_country] += match['RunsDescending']
    if not match['Not Out']:
        country_counts[host_country] += 1

# Calculate batting average for each country
country_averages = {}
for country in country_totals:
    if country_counts[country] > 0:
        country_averages[country] = country_totals[country] / country_counts[country]
    else:
        country_averages[country] = 0

# Print batting averages for each country
print(f"\n{player_name}'s batting averages in each country:")
for country, average in country_averages.items():
    print(f"{country}: {average:.2f}")


Enter the player's name: Q de Kock

Q de Kock's batting averages in each country:
West Indies: 118.50
South Africa: 41.89
India: 26.00
Australia: 56.20
New Zealand: 52.50
Zimbabwe: 81.00
England: 23.12
Sri Lanka: 22.12
Pakistan: 11.50
Bangladesh: 0.00


In [31]:
# Convert 'Inns' column to integer
df_batting['Inns'] = pd.to_numeric(df_batting['Inns'])

# Prompt the user to enter the player's name
player_name = input("Enter the player's name: ")

# Filter data for the player
player_data = df_batting[df_batting['Player'] == player_name]

# Calculate the total runs and times out for each innings
innings_totals = {}
innings_counts = {}

for innings in range(1, 5):
    innings_data = player_data[player_data['Inns'] == innings]
    total_runs = innings_data['RunsDescending'].sum()
    times_out = (1 - innings_data['Not Out']).sum()
    innings_totals[innings] = total_runs
    innings_counts[innings] = times_out

# Calculate batting average for each innings
innings_averages = {}
for innings, total_runs in innings_totals.items():
    if innings_counts[innings] > 0:
        innings_averages[innings] = total_runs / innings_counts[innings]
    else:
        innings_averages[innings] = 0

# Print batting averages for each innings
print(f"\n{player_name}'s batting averages based on innings:")
for innings, average in innings_averages.items():
    print(f"Innings {innings}: {average:.2f}")


Enter the player's name: Q de Kock

Q de Kock's batting averages based on innings:
Innings 1: 53.96
Innings 2: 43.44
Innings 3: 28.13
Innings 4: 20.92


In [82]:
# Step 1: Filter df_bowling for player and minimum wickets
min_wickets = int(input("Enter the minimum wickets: "))
player_name = input("Enter the player's name: ")

player_matches = df_bowling[(df_bowling['Player'] == player_name) & (df_bowling['WktsDescending'] >= min_wickets)]

# Step 2: Extract unique matches where the player took the minimum wickets or more
player_matches_ids = player_matches['NumericMatchID'].unique()

# Step 3: Filter df_team for matches where the player's team played
team_matches = df_team[df_team['NumericMatchID'].isin(player_matches_ids)]

# Step 4: Count the number of matches won, lost, and drawn by the player's team
team_won_matches_count = team_matches[(team_matches['Team'] == player_matches.iloc[0]['Team']) & (team_matches['Result'] == 'won')]['NumericMatchID'].nunique()
team_lost_matches_count = team_matches[(team_matches['Team'] == player_matches.iloc[0]['Team']) & (team_matches['Result'] == 'lost')]['NumericMatchID'].nunique()
team_drawn_matches_count = team_matches[(team_matches['Team'] == player_matches.iloc[0]['Team']) & (team_matches['Result'] == 'draw')]['NumericMatchID'].nunique()

# Step 5: Calculate the winning percentage
total_matches = len(player_matches_ids)

if total_matches == 0:
    print(f"No matches found for {player_name} with wickets greater than or equal to {min_wickets}.")
else:
    winning_percentage = (team_won_matches_count / total_matches) * 100
    losing_percentage = (team_lost_matches_count / total_matches) * 100
    drawing_percentage = (team_drawn_matches_count / total_matches) * 100
    print(f"Total matches where {player_name} took {min_wickets} or more wickets: {total_matches}")
    print(f"Matches won by {player_name}'s team: {team_won_matches_count}")
    print(f"Matches lost by {player_name}'s team: {team_lost_matches_count}")
    print(f"Matches drawn by {player_name}'s team: {team_drawn_matches_count}")
    print(f"The winning percentage for {player_name}'s team (minimum wickets: {min_wickets}) is {winning_percentage:.2f}%")
    print(f"The losing percentage for {player_name}'s team (minimum wickets: {min_wickets}) is {losing_percentage:.2f}%")
    print(f"The drawing percentage for {player_name}'s team (minimum wickets: {min_wickets}) is {drawing_percentage:.2f}%")


Enter the minimum wickets: 4
Enter the player's name: K Rabada
Total matches where K Rabada took 4 or more wickets: 22
Matches won by K Rabada's team: 14
Matches lost by K Rabada's team: 7
Matches drawn by K Rabada's team: 1
The winning percentage for K Rabada's team (minimum wickets: 4) is 63.64%
The losing percentage for K Rabada's team (minimum wickets: 4) is 31.82%
The drawing percentage for K Rabada's team (minimum wickets: 4) is 4.55%


In [34]:
# Prompt the user to enter the player's name
player_name = input("Enter the player's name: ")

# Filter data for the player
player_data = df_bowling[df_bowling['Player'] == player_name]

# Calculate the overall bowling average for the player
total_runs_conceded = player_data['Runs'].sum()
total_wickets_taken = player_data['WktsDescending'].sum()

if total_wickets_taken > 0:
    overall_bowling_average = total_runs_conceded / total_wickets_taken
else:
    overall_bowling_average = 0

print(f"{player_name}'s overall bowling average is:", overall_bowling_average)

# Calculate the average against each opposition
opposition_averages = {}
opposition_wickets = {}
for opposition, opposition_data in player_data.groupby('Opposition'):
    opposition_runs_conceded = opposition_data['Runs'].sum()
    opposition_wickets_taken = opposition_data['WktsDescending'].sum()
    if opposition_wickets_taken > 0:
        opposition_average = opposition_runs_conceded / opposition_wickets_taken
    else:
        opposition_average = 0
    opposition_averages[opposition] = opposition_average
    opposition_wickets[opposition] = opposition_wickets_taken

print(f"\n{player_name}'s bowling averages against each opposition:")
for opposition, average in opposition_averages.items():
    wickets = opposition_wickets[opposition]
    print(f"{opposition}: Average: {average:.2f}, Wickets: {wickets}")


Enter the player's name: K Rabada
K Rabada's overall bowling average is: 22.34642857142857

K Rabada's bowling averages against each opposition:
Australia: Average: 23.08, Wickets: 49
Bangladesh: Average: 12.00, Wickets: 15
England: Average: 25.29, Wickets: 66
India: Average: 24.57, Wickets: 44
New Zealand: Average: 28.09, Wickets: 23
Pakistan: Average: 23.41, Wickets: 22
Sri Lanka: Average: 21.51, Wickets: 35
West Indies: Average: 11.22, Wickets: 23
Zimbabwe: Average: 8.00, Wickets: 3


In [36]:
# Prompt the user to enter the player's name
player_name = input("Enter the player's name: ")

# Filter data for the player
player_data = df_bowling[df_bowling['Player'] == player_name]

# Create a dictionary to store total runs and total wickets taken for each country
country_runs = {}
country_wickets = {}

# Iterate through each match the player played
for index, match in player_data.iterrows():
    match_id = match['NumericMatchID']
    # Find the host country for the match
    host_country = df_team[df_team['NumericMatchID'] == match_id]['Host'].iloc[0]
    # Add runs and wickets to the corresponding country in the dictionary
    if host_country not in country_runs:
        country_runs[host_country] = 0
        country_wickets[host_country] = 0
    country_runs[host_country] += match['Runs']
    country_wickets[host_country] += match['WktsDescending']

# Calculate bowling average for each country
country_averages = {}
for country in country_runs:
    if country_wickets[country] > 0:
        country_averages[country] = country_runs[country] / country_wickets[country]
    else:
        country_averages[country] = 0

# Print bowling averages for each country
print(f"\n{player_name}'s bowling averages in each country:")
for country, average in country_averages.items():
    print(f"{country}: {average:.2f}")


Enter the player's name: DW Steyn

DW Steyn's bowling averages in each country:
India: 21.38
South Africa: 21.62
West Indies: 18.13
Zimbabwe: 10.50
Sri Lanka: 30.35
Pakistan: 29.61
England: 31.65
Australia: 28.77
Bangladesh: 14.20
New Zealand: 26.56


In [84]:
# Prompt user to enter the batsman's name
batsman_name = input("Enter batsman's name: ")

# Prompt user to enter the bowler's name
bowler_name = input("Enter bowler's name: ")

# Filter batting DataFrame for matches where the batsman played
df_batsman = df_batting[df_batting['Player'] == batsman_name]

# Ensure bowler information exists (if provided)
if bowler_name:
  if not any(df_bowling['Player'] == bowler_name):
    print(f"Bowler '{bowler_name}' not found in data. Please try again.")
    exit()

# Get batsman's team
batsman_team = df_batsman.iloc[0]['Team']  # Assuming Team info is present in the first row

# Get bowler's team (if applicable)
bowler_team = None
if bowler_name:
  bowler_team = df_bowling[df_bowling['Player'] == bowler_name]['Team'].iloc[0]

# Filter batting DataFrame for matches against the bowler's team (if provided)
df_batsman_opposition = df_batsman[df_batsman['Opposition'] == bowler_team] if bowler_team else df_batsman.copy()

# Calculate overall batting average against the opposition
batsman_average_overall = df_batsman_opposition['RunsDescending'].sum() / (1 - df_batsman_opposition['Not Out']).sum() if (1 - df_batsman_opposition['Not Out']).sum() > 0 else 0

# Filter for matches where the bowler was present (if applicable)
if bowler_name:
    matches_with_bowler = df_batsman_opposition['NumericMatchID'].isin(df_bowling[df_bowling['Player'] == bowler_name]['NumericMatchID'])
    df_batsman_with_bowler = df_batsman_opposition[matches_with_bowler]
else:
    df_batsman_with_bowler = df_batsman_opposition.copy()

# Calculate batting average against the bowler (if applicable)
if bowler_name:
    batsman_average_with_bowler = df_batsman_with_bowler['RunsDescending'].sum() / (1 - df_batsman_with_bowler['Not Out']).sum() if (1 - df_batsman_with_bowler['Not Out']).sum() > 0 else 0
else:
    batsman_average_with_bowler = None

# Calculate batting average against the opposition (excluding matches with the bowler)
df_batsman_without_bowler = df_batsman_opposition[~matches_with_bowler] if bowler_name else df_batsman_opposition.iloc[0:0]  # Empty dataframe if no bowler

batsman_average_without_bowler = df_batsman_without_bowler['RunsDescending'].sum() / (1 - df_batsman_without_bowler['Not Out']).sum() if (1 - df_batsman_without_bowler['Not Out']).sum() > 0 else 0

# Print the results
print(f"Overall batting average of {batsman_name} against {bowler_team if bowler_team else 'all teams'}: {batsman_average_overall}")

if bowler_name:
    print(f"Batting average of {batsman_name} when {bowler_name} was bowling: {batsman_average_with_bowler}")
    print(f"Batting average of {batsman_name} against {bowler_team} (excluding matches with {bowler_name}): {batsman_average_without_bowler}")


Enter batsman's name: V Kohli
Enter bowler's name: 
Overall batting average of V Kohli against all teams: 49.29545454545455


In [85]:
# Calculate win-loss ratio and sort teams
df_team['Outcome'] = df_team['Result'].map({'won': 'Win', 'lost': 'Loss', 'draw': 'Draw'})
team_outcome_counts = df_team.groupby(['Team', 'Outcome']).size().unstack(fill_value=0)
team_outcome_counts['Win-Loss Ratio'] = team_outcome_counts['Win'] / team_outcome_counts['Loss'].replace(0, 1)
sorted_teams = team_outcome_counts.sort_values(by='Win-Loss Ratio', ascending=False)

# Create bar plot with hover information
fig = px.bar(sorted_teams, x=sorted_teams.index, y='Win-Loss Ratio', title='Teams Win/Loss Ratio',
             labels={'index': 'Team', 'Win-Loss Ratio': 'Win/Loss Ratio'})

# Add hover information for wins and losses
fig.update_traces(hovertemplate='Team: %{x}<br>Win/Loss Ratio: %{y:.2f}<br>Wins: %{customdata[0]}<br>Losses: %{customdata[1]}',
                  customdata=sorted_teams[['Win', 'Loss']].values)

# Show the plot
fig.show()


In [39]:
import pandas as pd

# Prompt user to enter the batsman's name
batsman_name = input("Enter batsman's name: ")

# Prompt user to enter the bowler's name
bowler_name = input("Enter bowler's name: ")

# Filter batting DataFrame for matches where the batsman played
df_batsman = df_batting[df_batting['Player'] == batsman_name]

# Ensure bowler information exists (if provided)
if bowler_name:
    if not any(df_bowling['Player'] == bowler_name):
        print(f"Bowler '{bowler_name}' not found in data. Please try again.")
        exit()

# Get batsman's team
batsman_team = df_batsman.iloc[0]['Team']  # Assuming Team info is present in the first row

# Get bowler's team (if applicable)
bowler_team = None
if bowler_name:
    bowler_team = df_bowling[df_bowling['Player'] == bowler_name]['Team'].iloc[0]

# Filter batting DataFrame for matches against the bowler's team (if provided)
df_batsman_opposition = df_batsman[df_batsman['Opposition'] == bowler_team] if bowler_team else df_batsman.copy()

# Calculate overall batting average against the opposition
batsman_average_overall = df_batsman_opposition['RunsDescending'].sum() / (1 - df_batsman_opposition['Not Out']).sum() if (1 - df_batsman_opposition['Not Out']).sum() > 0 else 0

# Filter for matches where the bowler was present (if applicable)
if bowler_name:
    matches_with_bowler = df_batsman_opposition['NumericMatchID'].isin(df_bowling[df_bowling['Player'] == bowler_name]['NumericMatchID'])
    df_batsman_with_bowler = df_batsman_opposition[matches_with_bowler]
else:
    df_batsman_with_bowler = df_batsman_opposition.copy()

# Calculate batting average against the bowler (if applicable)
if bowler_name:
    batsman_average_with_bowler = df_batsman_with_bowler['RunsDescending'].sum() / (1 - df_batsman_with_bowler['Not Out']).sum() if (1 - df_batsman_with_bowler['Not Out']).sum() > 0 else 0
else:
    batsman_average_with_bowler = None

# Calculate batting average against the opposition (excluding matches with the bowler)
df_batsman_without_bowler = df_batsman_opposition[~matches_with_bowler] if bowler_name else df_batsman_opposition.iloc[0:0]  # Empty dataframe if no bowler

batsman_average_without_bowler = df_batsman_without_bowler['RunsDescending'].sum() / (1 - df_batsman_without_bowler['Not Out']).sum() if (1 - df_batsman_without_bowler['Not Out']).sum() > 0 else 0

# Create a DataFrame for the results
data = {
    'Metric': ['Overall Batting Average', 'Batting Average with Bowler', 'Batting Average without Bowler'],
    'Value': [batsman_average_overall, batsman_average_with_bowler, batsman_average_without_bowler]
}
result_df = pd.DataFrame(data)

# Print the results as a table
print("Results:")
print(result_df)


Enter batsman's name: DA Warner
Enter bowler's name: K Rabada
Results:
                           Metric      Value
0         Overall Batting Average  52.407407
1     Batting Average with Bowler  41.625000
2  Batting Average without Bowler  68.090909
